In [18]:
# https://pypi.org/project/exif/
# pip install exif

# https://pypi.org/project/pymediainfo/
# pip install pymediainfo

In [66]:
import pandas as pd
# import PIL.Image
from exif import Image
from pprint import pprint
from pymediainfo import MediaInfo
from pathlib import Path
import numpy as np
import os
import re
import numpy as np
import json

In [98]:
# path = Path(r'G:\Google Photo_extr\Takeout\Google Фото')
# path = Path(r'D:\Рабочая\!_Scripts\change_file_creation_time')
# path = Path('G:/temp/Тест фото')
path = Path('E:\Тест фото')

In [10]:
# nef_path = df.query('extension == ".nef"').path

In [9]:
# nef_path.loc[15967]

In [42]:
def get_photo_exif_date(full_path):
    """
    функция получает на вход полный путь к файлу картинки и возвращает кортеж из трёх дат получинных из exif:
        (datetime, datetime_original, datetime_digitized)
    """
    with open(full_path, 'rb') as image_file:
        my_image = Image(image_file)
    
    # считываем 3 даты из файла
    datetime = my_image.datetime
    datetime_original  = my_image.datetime_original
    datetime_digitized = my_image.datetime_digitized
    
    # паттерн для перевода дат в нужный формат (дата) (время)
    fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
    
    # переводим даты в формат времени по шаблону выше
    datetime = pd.to_datetime(
                fpattern
                    .findall(datetime)[0][0]
                    .replace(':','-') + ' ' + fpattern
                    .findall(datetime)[0][1]
               )
    datetime_original = pd.to_datetime(
                fpattern
                    .findall(datetime_original)[0][0]
                    .replace(':','-') + ' ' + fpattern
                    .findall(datetime_original)[0][1]
               )
    datetime_digitized = pd.to_datetime(
                fpattern
                    .findall(datetime_digitized)[0][0]
                    .replace(':','-') + ' ' + fpattern
                    .findall(datetime_digitized)[0][1]
                )
    return datetime, datetime_original, datetime_digitized

In [109]:
def get_video_exif_date(full_path):
    """
    функция получает на вход полный путь к файлу и возвращает дату создания
    """
    media_info = MediaInfo.parse(full_path)
    for track in media_info.tracks:
        if track.track_type == "Video":
            exif_vdata = track.to_data()
    vdata = exif_vdata['tagged_date']

    # часовой пояс
    time_zone = pd.to_timedelta(7, unit='h')

    vdata = pd.to_datetime(vdata[4:]) + time_zone
    return vdata

In [63]:
# ищем год в названии файла
def get_year_name(full_path):
    pattern = re.compile('(20\d\d)[^0-9]')
    years = pattern.findall(full_path)
    if len(years) > 0:
        return ' '.join(years)
    else:
        return np.nan

In [93]:
# извлекаем дату из json файла
def get_json_date(full_path):
    try:
        json_path = str(full_path) + '.json'
        with open(json_path) as json_file:
            json_data = json.load(json_file)
        
        # часовой пояс
        time_zone = pd.to_timedelta(7, unit='h')
        
        # извлекаем время из json, два варианта
        video_creation_time = pd.to_datetime(json_data["creationTime"]['timestamp'], unit='s') + time_zone
        video_taken_time    = pd.to_datetime(json_data["photoTakenTime"]["timestamp"], unit='s') + time_zone
        return video_creation_time, video_taken_time
    except:
        return np.nan, np.nan


In [110]:
# удалить
# функция вытаскивающая exif
def get_exif(path_col):
    
    # списки расширений для видео и фото
    video_ext = np.array(['.mp4', '.mts', '.avi', '.mov', '.3gp', '.mpg', '.mkv'])
    photo_ext = np.array(['.jpg', '.jpeg', '.nef', '.gif', '.png', '.bmp'])
    
    # вытаскиваем exif из видео
    if path_col.suffix.lower() in video_ext:
        try:
            media_info = MediaInfo.parse(path_col)
            for track in media_info.tracks:
                if track.track_type == "Video":
                    exif_vdata = track.to_data()
            vdata = exif_vdata['tagged_date']
            vdata = pd.to_datetime(vdata[4:])
            time_zone = pd.to_timedelta(7, unit='h')
            return pd.to_datetime(vdata[4:]) + time_zone
        except:
            return np.nan()
    
    # вытаскиваем exif из фото
    elif path_col.suffix.lower() in photo_ext :
        try:
            with open('IMG_20211123_162603.jpg', 'rb') as image_file:
                my_image = Image(image_file)
            fdata = my_image.datetime
            fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
            return pd.to_datetime(fpattern.findall(fdata)[0][0].replace(':','-') + ' ' + fpattern.findall(fdata)[0][1])
        except:
            return np.nan
    else:
        return np.nan

In [3]:
%%time
df = pd.DataFrame(columns=['name', 'full_name', 'extension', 'path'])
for file_path in path.glob('**/*.*'):
    if file_path.is_file():
        extension = file_path.suffix.lower()
        name = Path(file_path.stem).stem
        full_name = file_path.name
        parent_path = file_path.parent
        df.loc[len(df)] = [name, full_name, extension, parent_path]
df.head()

Wall time: 52 ms


,name,full_name,extension,path
0,IMG_0064,IMG_0064.JPG,.jpg,E:\Тест фото\Алтайталь
1,IMG_0064,IMG_0064.JPG.json,.json,E:\Тест фото\Алтайталь
2,IMG_0065,IMG_0065.JPG,.jpg,E:\Тест фото\Алтайталь
3,IMG_0065,IMG_0065.JPG.json,.json,E:\Тест фото\Алтайталь
4,IMG_0272,IMG_0272.JPG,.jpg,E:\Тест фото\Алтайталь


In [102]:
len(df)

0

In [122]:
# списки расширений для видео и фото
video_ext = np.array(['.mp4', '.mts', '.avi', '.mov', '.3gp', '.mpg', '.mkv'])
photo_ext = np.array(['.jpg', '.jpeg', '.nef', '.gif', '.png', '.bmp'])

In [144]:
%%time
df = pd.DataFrame(columns=['name', 'extension', 'path', 
                           'video_exif_date', 
                           'photo_date', 'photo_date_orig', 'photo_date_dig', 
                           'json_creation_time', 'json_taken_time',
                           'year_from_name'])

for file_path in path.glob('**/*.*'):
    # условие, чтобы элемент был файлом с расшерением видео или фото
    if file_path.is_file() and (file_path.suffix.lower() in video_ext or file_path.suffix.lower() in photo_ext):
        video_exif_date = np.nan
        photo_date, photo_date_orig, photo_date_dig = np.nan, np.nan, np.nan
        # имя файла
        name = file_path.stem
        # расшерение файла
        extension = file_path.suffix.lower()
        
        # если видео
        if file_path.suffix.lower() in video_ext:
            video_exif_date = get_video_exif_date(file_path)
            # из json
            json_creation_time, json_taken_time = get_json_date(file_path)
            # из названия файла
            year_from_name = get_year_name(name)
            
        # если фото
        elif file_path.suffix.lower() in photo_ext:
            photo_date, photo_date_orig, photo_date_dig = get_photo_exif_date(file_path)
            # из json
            json_creation_time, json_taken_time = get_json_date(file_path)
            # из названия файла
            year_from_name = get_year_name(name)

        df.loc[len(df)] = [name, extension, file_path, 
                           video_exif_date, 
                           photo_date, photo_date_orig, photo_date_dig, 
                           json_creation_time, json_taken_time,
                           year_from_name]

Wall time: 56 ms


In [145]:
df

,name,extension,path,video_exif_date,photo_date,photo_date_orig,photo_date_dig,json_creation_time,json_taken_time,year_from_name
0,IMG_0064,.jpg,E:\Тест фото\Алтайталь\IMG_0064.JPG,NaN,2006-08-21 11:29:57,2006-08-21 11:29:57,2006-08-21 11:29:57,2019-02-28 22:19:01,2006-08-21 10:29:57,NaN
1,IMG_0065,.jpg,E:\Тест фото\Алтайталь\IMG_0065.JPG,NaN,2006-08-21 11:30:28,2006-08-21 11:30:28,2006-08-21 11:30:28,2019-02-28 22:19:01,2006-08-21 10:30:28,NaN
2,IMG_0272,.jpg,E:\Тест фото\Алтайталь\IMG_0272.JPG,NaN,2006-08-11 09:03:14,2006-08-11 09:03:14,2006-08-11 09:03:14,2019-02-28 22:19:01,2006-08-11 08:03:14,NaN
3,IMG_0273,.jpg,E:\Тест фото\Алтайталь\IMG_0273.JPG,NaN,2006-08-11 09:03:38,2006-08-11 09:03:38,2006-08-11 09:03:38,2019-02-28 22:19:01,2006-08-11 08:03:38,NaN
4,IMG_0276,.jpg,E:\Тест фото\Алтайталь\IMG_0276.JPG,NaN,2006-08-11 09:34:10,2006-08-11 09:34:10,2006-08-11 09:34:10,2019-02-28 22:19:01,2006-08-11 08:34:10,NaN
5,IMG_0277,.jpg,E:\Тест фото\Алтайталь\IMG_0277.JPG,NaN,2006-08-11 09:34:30,2006-08-11 09:34:30,2006-08-11 09:34:30,2019-02-28 22:19:01,2006-08-11 08:34:30,NaN
6,IMG_0279,.jpg,E:\Тест фото\Алтайталь\IMG_0279.JPG,NaN,2006-08-11 09:35:22,2006-08-11 09:35:22,2006-08-11 09:35:22,2019-02-28 22:19:01,2006-08-11 08:35:22,NaN
7,IMG_0280,.jpg,E:\Тест фото\Алтайталь\IMG_0280.JPG,NaN,2006-08-11 09:35:55,2006-08-11 09:35:55,2006-08-11 09:35:55,2019-02-28 22:19:01,2006-08-11 08:35:55,NaN
8,IMG_0282,.jpg,E:\Тест фото\Алтайталь\IMG_0282.JPG,NaN,2006-08-11 09:39:26,2006-08-11 09:39:26,2006-08-11 09:39:26,2019-02-28 22:19:01,2006-08-11 08:39:26,NaN
9,Горный 2015_720p,.mp4,E:\Тест фото\Видео\Горный 2015_720p.mp4,2016-07-30 21:11:13,NaT,NaT,NaT,2016-07-30 18:45:24,2016-03-19 14:58:46,2015


In [141]:
df.dtypes

name                          object
extension                     object
path                          object
video_exif_date               object
photo_date            datetime64[ns]
photo_date_orig       datetime64[ns]
photo_date_dig        datetime64[ns]
json_creation_time    datetime64[ns]
json_taken_time       datetime64[ns]
year_from_name                object
dtype: object

In [135]:
datetime_digitized

Timestamp('2006-08-21 11:29:57')

In [116]:
photo_ext

array(['.jpg', '.jpeg', '.nef', '.gif', '.png', '.bmp'], dtype='<U5')

In [105]:
df

,name,extension,path
0,IMG_0064,.jpg,E:\Тест фото\Алтайталь\IMG_0064.JPG
1,IMG_0064.JPG,.json,E:\Тест фото\Алтайталь\IMG_0064.JPG.json
2,IMG_0065,.jpg,E:\Тест фото\Алтайталь\IMG_0065.JPG
3,IMG_0065.JPG,.json,E:\Тест фото\Алтайталь\IMG_0065.JPG.json
4,IMG_0272,.jpg,E:\Тест фото\Алтайталь\IMG_0272.JPG
5,IMG_0272.JPG,.json,E:\Тест фото\Алтайталь\IMG_0272.JPG.json
6,IMG_0273,.jpg,E:\Тест фото\Алтайталь\IMG_0273.JPG
7,IMG_0273.JPG,.json,E:\Тест фото\Алтайталь\IMG_0273.JPG.json
8,IMG_0276,.jpg,E:\Тест фото\Алтайталь\IMG_0276.JPG
9,IMG_0276.JPG,.json,E:\Тест фото\Алтайталь\IMG_0276.JPG.json


In [72]:
# список всех методов
# my_image.list_all()

In [63]:
fdata = my_image.datetime
fdata

'2021:11:23 16:26:05'

In [71]:
fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
fpattern.findall(fdata)[0][0].replace(':','-') + ' ' + fpattern.findall(fdata)[0][1]

'2021-11-23 16:26:05'

In [25]:
my_image.datetime_original

'2021:11:23 16:26:05'

In [26]:
my_image.datetime_digitized

'2021:11:23 16:26:05'

In [27]:
my_image.gps_datestamp

'2021:11:23'